# MNIST Classification on Sagemaker using Apache MXNet Module API

The **SageMaker Python SDK** makes it easy to train and deploy MXNet models. In this assignment, I have used neural network using the Apache MXNet [Module API](https://mxnet.incubator.apache.org/api/python/module.html) and the MNIST dataset. This notebook is basically to explore and understand how to save and deploy a trained machine learning model. I have also included a TensorFlow code to build a model on the same dataset and even that can be used to deploy an API. However, my learning is that it took me several days to build a model from scratch using TensorFlow, however it was extremely easy and convenient to use this ot build or deploy the model and the accuracy is pretty awesome (99.8%) 

**About dataset** The MNIST dataset is widely used for handwritten digit classification, and consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). The task at hand is to train a model using the 60,000 training images and subsequently test its classification accuracy on the 10,000 test images.

### Loading dependencies and basic setup

First we need to define a few variables that will be needed later.

In [1]:
from sagemaker import get_execution_role

#Bucket location to save custom code in tar.gz format
custom_code_upload_location = 's3://007-sagemaker-mnist/customcode/mxnet'

#Bucket location where results of model training are saved
model_artifacts_location = 's3://007-sagemaker-mnist/artifacts'

role = get_execution_role()

### Training script

The ``mnist.py`` script provides all the code for training and hosting a SageMaker model. Source - https://mxnet.incubator.apache.org/tutorials/python/mnist.html

In [2]:
!cat mnist.py

import logging

import gzip
import mxnet as mx
import numpy as np
import os
import struct


def load_data(path):
    with gzip.open(find_file(path, "labels.gz")) as flbl:
        struct.unpack(">II", flbl.read(8))
        labels = np.fromstring(flbl.read(), dtype=np.int8)
    with gzip.open(find_file(path, "images.gz")) as fimg:
        _, _, rows, cols = struct.unpack(">IIII", fimg.read(16))
        images = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(labels), rows, cols)
        images = images.reshape(images.shape[0], 1, 28, 28).astype(np.float32) / 255
    return labels, images


def find_file(root_path, file_name):
    for root, dirs, files in os.walk(root_path):
        if file_name in files:
            return os.path.join(root, file_name)


def build_graph():
    data = mx.sym.var('data')
    data = mx.sym.flatten(data=data)
    fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
    act1 = mx.sym.Activation(data=fc1, act_type="relu")
    fc2 = mx.sym.FullyConnect

### SageMaker's MXNet estimator class

The SageMaker ```MXNet``` estimator allows us to run single machine or distributed training in SageMaker, using CPU or GPU-based instances. When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role, and the S3 locations we defined in the setup section. We also provide a few other parameters. ``train_instance_count`` and ``train_instance_type`` determine the number and type of SageMaker instances that will be used for the training job. The ``hyperparameters`` parameter is a ``dict`` of values that will be passed to your training script -- you can see how to access these values in the ``mnist.py`` script above.

In [3]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.1})

### Running the Training Job

After we've constructed our MXNet object, we can fit it using data stored in S3. Below we run SageMaker training on two input channels: **train** and **test**.

During training, SageMaker makes this data stored in S3 available in the local filesystem where the mnist script is running. The ```mnist.py``` script simply loads the train and test data from disk.

In [4]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-py2-cpu-2018-03-10-02-49-51-481


.....................................................................
executing startup script (first run)
2018-03-10 02:55:07,708 INFO - root - running container entrypoint
2018-03-10 02:55:07,709 INFO - root - starting train task
2018-03-10 02:55:09,501 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 0, 'channels': {u'test': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'train': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'hosts': [u'algo-1']}, 'user_script_name': u'mnist.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'test': u'/opt/ml/input/data/test', u'train': u'/opt/ml/input/data/train'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/output', 'model_dir': '/opt

2018-03-10 02:56:01,167 INFO - root - Epoch[24] Train-accuracy=0.998283
2018-03-10 02:56:01,167 INFO - root - Epoch[24] Time cost=1.956
2018-03-10 02:56:01,314 INFO - root - Epoch[24] Validation-accuracy=0.977000
===== Job Complete =====
CPU times: user 468 ms, sys: 40 ms, total: 508 ms
Wall time: 7min 10s


### Creating an inference Endpoint

After training, we use the ``MXNet estimator`` object to build and deploy an ``MXNetPredictor``. This creates a Sagemaker **Endpoint** -- a hosted prediction service that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single ``ml.m4.xlarge`` instance.

In [5]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-py2-cpu-2018-03-10-02-49-51-481
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-py2-cpu-2018-03-10-02-49-51-481


-------------------------------------------------------------------------------------------------!CPU times: user 408 ms, sys: 32 ms, total: 440 ms
Wall time: 8min 27s


The request handling behavior of the Endpoint is determined by the ``mnist.py`` script. In this case, the script doesn't include any request handling functions, so the Endpoint will use the default handlers provided by SageMaker. These default handlers allow us to perform inference on input data encoded as a multi-dimensional JSON array.

### Making an inference request

Now that the Endpoint is deployed and we have a ``predictor`` object, we can use it to classify handwritten digits.

In [11]:
from IPython.display import HTML
HTML(open("input.html").read())

Now we can use the ``predictor`` object to classify the handwritten digit:

In [13]:
response = predictor.predict(data)
print('Raw prediction result:')
print(response)

labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Raw prediction result:
[[7.20521597941115e-08, 1.7779884231572396e-08, 0.9999977350234985, 8.220210929721361e-07, 1.6636187869997765e-14, 1.4069667306215727e-11, 5.25204379986377e-10, 1.182909045382985e-06, 9.200454798019564e-08, 4.4998726966838376e-11]]
Labeled predictions: 
[(0, 7.20521597941115e-08), (1, 1.7779884231572396e-08), (2, 0.9999977350234985), (3, 8.220210929721361e-07), (4, 1.6636187869997765e-14), (5, 1.4069667306215727e-11), (6, 5.25204379986377e-10), (7, 1.182909045382985e-06), (8, 9.200454798019564e-08), (9, 4.4998726966838376e-11)]
Most likely answer: (2, 0.9999977350234985)


# Deleting the Endpoint

Deleting the prediction endpoint to release the instance(s) associated with it on AWS.

In [15]:
print("Endpoint name: " + predictor.endpoint)

Endpoint name: sagemaker-mxnet-py2-cpu-2018-03-10-02-49-51-481


In [16]:
import sagemaker

sagemaker.Session().delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-mxnet-py2-cpu-2018-03-10-02-49-51-481
